# Art Gallery Problem

## Geometrical Helper Classes

In [1]:
# Point: (x|y)
class Point:
    def __init__(self,x,y):
        self.x = x
        self.y = y
    def __repr__(self):
        return f"{self.__class__.__name__}(x={self.x},y={self.y})"
    def __str__(self):
        return f"({self.x}|{self.y})"

In [2]:
# Line with equation: ax + by = c
class Line:
    def __init__(self,a,b,c):
        self.a = a
        self.b = b
        self.c = c
    def __repr__(self):
        return f"{self.__class__.__name__}(a={self.a},b={self.b},c={self.c})"
    def __str__(self):
        return f"{self.a}x + {self.b}y = {self.c}"

### Line Segment
Next, we take a look at line segments. The class LineSegment will be subclass of Line, but instead of the paramters $a$, $b$ and $c$ it will take the starting and endpoint of the line segment as parameters.

#### How to calculate the paramaters?
We want to calculate the equation $ax+by=c$ of the line passing through our line segment. Given the starting point $(x_0|y_0)$ and the endpoint $(x_1|y_1)$, we can calculate the slope of the line as
$$a=\frac{y_1-y_0}{x_1-x_0}.$$
Prior to this calculation it has to be checked though that $x_0\neq x_1$.

In [3]:
# Line segment from Point start to Point end
class LineSegment(Line):
    def __init__(self,start,end):
        self.start = start
        self.end = end
        
        
    def __repr__(self):
        return f"{self.__class__.__name__}(start={repr(self.start)},end={repr(self.end)})"
    def __str__(self):
        return f"{self.start} <-> {self.end}"